In [0]:
#importing the tools
from env import env
from src import utils, excel

import openpyxl
import pandas as pd

from pyspark.sql import functions as F
from datetime import datetime
from pyspark.sql.functions import date_format
from openpyxl.styles import NamedStyle

In [0]:
# Load the SA data
df_raw_sa = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(env["spec_advi_path"])
wb = openpyxl.load_workbook('ICB Template.xlsx')
report_start = 'April 2022 to '

#display (spec_advi_path)


publishing_month = df_raw_sa.select(F.max("EROC_DerMonth")).collect()[0][0]
publishing_month = datetime.strptime(publishing_month, '%Y-%m-%d')
publishing_month = publishing_month.strftime("%B %Y")
date_header = (report_start + publishing_month) 

#print(date_header)
#display(df_raw_sa)

In [0]:
#Total requests for all types of SA
df_icb_total_all_types_sa = df_raw_sa.where((F.col("EROC_DerMonth") > '2022-03-01')) #& (F.col("Acute_Status") == "Acute"))
    
df_icb_total_all_types_sa = df_icb_total_all_types_sa.groupby(
        "EROC_DerMonth",
        "EROC_STP_Code",
        "EROC_STP_Name"
    ).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
        "EROC_DerMonth",
        "EROC_STP_Code"
    )

# Select only the columns you need
df_icb_total_all_types_sa = df_icb_total_all_types_sa.select(
    "EROC_DerMonth",
    "EROC_STP_Code",
    "EROC_STP_Name",
    "Requests"
)

#display(df_icb_total_all_types_sa)

In [0]:
#putting the data in pivot table format
df_icb_total_all_types_sa_pivot = (df_icb_total_all_types_sa
    .groupby(
        "EROC_STP_Code",
        "EROC_STP_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "EROC_STP_Name",
        "EROC_STP_Code",
        )
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)

#display(df_icb_total_all_types_sa_pivot)

In [0]:
#1.Outputs of Total requests for all types sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_total_all_types_sa_pivot.columns[2:]:
    try:
        month_format = datetime.strptime(column, '%Y-%m-%d')
        month_format = month_format.strftime("%b-%Y")
        df_icb_total_all_types_sa_pivot = df_icb_total_all_types_sa_pivot.withColumnRenamed(column, month_format)
    except ValueError:
        continue

#2.converting the pivot to pandas dataframe
df_pd_icb_total_all_types_sa_pivot = df_icb_total_all_types_sa_pivot.toPandas()

#3.creating a workbook
ws_icb_total_all = wb['1) All Types of Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_total_all_types_sa_pivot,
    ws=ws_icb_total_all,
    header=True,
    startrow=15,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_total_all_types_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(15, 59):
        cell_to_copy_from = ws_icb_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_total_all_types_sa.select("EROC_STP_Name").distinct().count()
new_icb = number_of_icb - 42
pre_table_rows = 14 
copy_row = pre_table_rows + 42
end_row = copy_row + new_icb + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_icb_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D15"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

for rule in ws_icb_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        ws_icb_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_icb_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_icb_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format
        
# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_icb_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_icb_total_all.iter_rows(min_row=15, max_row=57, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
#icb_Processed requests for all types of SA
df_icb_processed_all_types_sa = df_raw_sa.where((F.col("EROC_DerMonth") > '2022-03-01') & 
                                            #(F.col("Acute_Status") == "Acute") & 
                                            (F.col("EROC_DerStatus") != 2) & 
                                            (F.col("EROC_DerOutcome") != 40))

df_icb_processed_all_types_sa = df_icb_processed_all_types_sa.groupby(
        "EROC_DerMonth",
        "EROC_STP_Code",
        "EROC_STP_Name"
    ).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
        "EROC_DerMonth",
        "EROC_STP_Code"
    )

# Select only the columns you need
df_icb_processed_all_types_sa = df_icb_processed_all_types_sa.select(
    "EROC_DerMonth",
    "EROC_STP_Code",
    "EROC_STP_Name",
    "Requests"
)

#display(df_icb_processed_all_types_sa)

In [0]:
#putting the data in pivot table format
df_icb_processed_all_types_sa_pivot = (df_icb_processed_all_types_sa
    .groupby(
        "EROC_STP_Code",
        "EROC_STP_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "EROC_STP_Name",
        "EROC_STP_Code",
        )
    .withColumnRenamed("EROC_STP_Code", "Provider Code")
    .withColumnRenamed("EROC_STP_Name", "Provider Name")
)

#display(df_icb_processed_all_types_sa_pivot)


In [0]:
#1.Outputs of processed requests for all types sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_processed_all_types_sa_pivot.columns[2:]:
    month_format = datetime.strptime(column, '%Y-%m-%d')
    month_format = month_format.strftime("%b-%Y")
    df_icb_processed_all_types_sa_pivot = df_icb_processed_all_types_sa_pivot.withColumnRenamed(column, month_format)

#2.converting the pivot to pandas dataframe
df_pd_icb_processed_all_types_sa_pivot = df_icb_processed_all_types_sa_pivot.toPandas()

#3.creating a workbook
ws_total_all = wb['1) All Types of Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_processed_all_types_sa_pivot,
    ws=ws_total_all,
    header=True,
    startrow=64,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_processed_all_types_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(64, 108):
        cell_to_copy_from = ws_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_processed_all_types_sa.select("EROC_STP_Name").distinct().count()
new_provider = number_of_icb - 42
pre_table_rows = 63 
copy_row = pre_table_rows + 42
end_row = copy_row + new_provider + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D64"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

rules_to_add = []
for rule in ws_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        rules_to_add.append(rule)

for rule in rules_to_add:
    ws_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format

# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_total_all.iter_rows(min_row=64, max_row=106, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
#icb Diverted requests for all types of SA 
df_icb_diverted_all_types_sa = df_raw_sa.where((F.col("EROC_DerMonth") > '2022-03-01') & 
                                            #(F.col("Acute_Status") == "Acute") & 
                                            (F.col("EROC_DerStatus") != 2) & 
                                            (F.col("EROC_DerOutcome").isin (10,12)))

df_icb_diverted_all_types_sa = df_icb_diverted_all_types_sa.groupby(
        "EROC_DerMonth",
        "EROC_STP_Code",
        "EROC_STP_Name"
    ).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
        "EROC_DerMonth",
        "EROC_STP_Code"
    )

# Select only the columns you need
df_icb_diverted_all_types_sa = df_icb_diverted_all_types_sa.select(
    "EROC_DerMonth",
    "EROC_STP_Code",
    "EROC_STP_Name",
    "Requests"
)

#display(df_icb_diverted_all_types_sa)

In [0]:
#putting the data in pivot table format
df_icb_diverted_all_types_sa_pivot = (df_icb_diverted_all_types_sa
    .groupby(
        "EROC_STP_Code",
        "EROC_STP_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "EROC_STP_Name",
        "EROC_STP_Code",
        )
    .withColumnRenamed("EROC_STP_Code", "Provider Code")
    .withColumnRenamed("EROC_STP_Name", "Provider Name")
)

#display(df_icb_diverted_all_types_sa_pivot)

In [0]:
#1.Outputs of diverted requests for all types sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_diverted_all_types_sa_pivot.columns[2:]:
    if column[:4].isdigit() and '-' in column:
        try:
            month_format = datetime.strptime(column, '%Y-%m-%d').strftime("%b-%Y")
            df_icb_diverted_all_types_sa_pivot = df_icb_diverted_all_types_sa_pivot.withColumnRenamed(column, month_format)
        except ValueError:
            pass

#2.converting the pivot to pandas dataframe
df_pd_icb_diverted_all_types_sa_pivot = df_icb_diverted_all_types_sa_pivot.toPandas()

#3.creating a workbook
ws_total_all = wb['1) All Types of Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_diverted_all_types_sa_pivot,
    ws=ws_total_all,
    header=True,
    startrow=112,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_diverted_all_types_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(112, 156):
        cell_to_copy_from = ws_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_diverted_all_types_sa.select("EROC_STP_Name").distinct().count()
new_provider = number_of_icb - 42
pre_table_rows = 111 
copy_row = pre_table_rows + 42
end_row = copy_row + new_provider + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D112"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

rules_to_add = []
for rule in ws_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        rules_to_add.append(rule)

for rule in rules_to_add:
    ws_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format

# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_total_all.iter_rows(min_row=112, max_row=154, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
# Total pre referral
from pyspark.sql.functions import col, to_date, lit, sum as F_sum

# Step 0: Ensure EROC_DerMonth is a date
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

# Step 1: Get all Acute providers
df_all_icb = (
    df_raw_sa  # .filter(col("Acute_Status") == "Acute")
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

# Step 2: Filter for pre-referral SA requests only
df_icb_total_pre_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        # (col("Acute_Status") == "Acute") &
        (col("EROC_DerTypeOfSpecialistAdvice") == 1)# &
        #(col("EROC_DerOutcome") != 40)
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

# Step 3: Group and aggregate
df_icb_total_pre_referral_sa = (
    df_icb_total_pre_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

# Step 4: Cross-join with all orgs and all months to preserve missing combinations

# Get all months in the dataset
all_months_df = (
    df_icb_total_pre_referral_sa
    .select("EROC_DerMonth")
    .distinct()
    .filter(col("EROC_DerMonth").isNotNull())
)

# DEBUG: Print all available months
# print(" Available Months (Before Pivot):")
all_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()
# print(all_months)

# Create full grid: all_orgs x all_months
full_grid = (
    df_all_icb.crossJoin(all_months_df)
)

# Left join aggregated data to preserve all org/month combos
df_icb_total_pre_referral_sa_full = (
    full_grid.join(
        df_icb_total_pre_referral_sa,
        on=["EROC_STP_Code", "EROC_STP_Name", "EROC_DerMonth"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

# Step 5: Pivot table with all valid months
valid_months = all_months  # Already collected above

df_icb_total_pre_referral_sa_pivot = (
    df_icb_total_pre_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)
#display(df_icb_total_pre_referral_sa_pivot)


In [0]:
#Ignoring any merged cells in template
from openpyxl.cell.cell import MergedCell

def safe_insert_pandas_df_into_excel(df, ws, header, startrow, startcol, index):
    """
    Inserts DataFrame into Excel sheet, skipping merged cells.
    """
    # Write headers if needed
    col_offset = 1 if index else 0
    if header:
        for j, col_name in enumerate(df.columns):
            cell = ws.cell(row=startrow, column=startcol + j + col_offset)
            if not isinstance(cell, MergedCell):
                cell.value = col_name

        startrow += 1  # move to next row for data

    # Write the data
    for i, row in enumerate(df.itertuples(index=index, name=None)):
        for j, val in enumerate(row):
            cell = ws.cell(row=startrow + i, column=startcol + j)
            if not isinstance(cell, MergedCell):
                cell.value = val

In [0]:
#1.Outputs of Total requests for all re referral sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_total_pre_referral_sa_pivot.columns[2:]:
    try:
        month_format = datetime.strptime(column, '%Y-%m-%d')
        month_format = month_format.strftime("%b-%Y")
        df_icb_total_pre_referral_sa_pivot = df_icb_total_pre_referral_sa_pivot.withColumnRenamed(column, month_format)
    except ValueError:
        continue

#2.converting the pivot to pandas dataframe
df_pd_icb_total_pre_referral_sa_pivot = df_icb_total_pre_referral_sa_pivot.toPandas()

#3.creating a workbook
ws_icb_total_all = wb['2) Pre referral Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_total_pre_referral_sa_pivot,
    ws=ws_icb_total_all,
    header=True,
    startrow=15,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_total_pre_referral_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(15, 59):
        cell_to_copy_from = ws_icb_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_total_pre_referral_sa.select("EROC_STP_Name").distinct().count()
new_icb = number_of_icb - 42
pre_table_rows = 14 
copy_row = pre_table_rows + 42
end_row = copy_row + new_icb + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_icb_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D15"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

for rule in ws_icb_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        ws_icb_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_icb_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_icb_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format
        
# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_icb_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_icb_total_all.iter_rows(min_row=15, max_row=57, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
# icb processed pre referral
from pyspark.sql.functions import col, to_date, lit, sum as F_sum

# Step 0: Ensure EROC_DerMonth is a date
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

# Step 1: Get all Acute providers
df_all_icb = (
    df_raw_sa  # .filter(col("Acute_Status") == "Acute")
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

# Step 2: Filter for pre-referral SA requests only
df_icb_processed_pre_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        (col("EROC_DerStatus") != 2) &
        (col("EROC_DerTypeOfSpecialistAdvice") == 1) &
        (col("EROC_DerOutcome") != 40)
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

# Step 3: Group and aggregate
df_icb_processed_pre_referral_sa = (
    df_icb_processed_pre_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

# Step 4: Cross-join with all orgs and all months to preserve missing combinations

# Get all months in the dataset
all_months_df = (
    df_icb_processed_pre_referral_sa
    .select("EROC_DerMonth")
    .distinct()
    .filter(col("EROC_DerMonth").isNotNull())
)

# DEBUG: Print all available months
# print(" Available Months (Before Pivot):")
all_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()
# print(all_months)

# Create full grid: all_orgs x all_months
full_grid = (
    df_all_icb.crossJoin(all_months_df)
)

# Left join aggregated data to preserve all org/month combos
df_icb_processed_pre_referral_sa_full = (
    full_grid.join(
        df_icb_processed_pre_referral_sa,
        on=["EROC_STP_Code", "EROC_STP_Name", "EROC_DerMonth"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

# Step 5: Pivot table with all valid months
valid_months = all_months  # Already collected above

df_icb_processed_pre_referral_sa_pivot = (
    df_icb_processed_pre_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)
#display(df_icb_processed_pre_referral_sa_pivot)

In [0]:
#1.Outputs of processed requests for pre referral sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_processed_pre_referral_sa_pivot.columns[2:]:
    month_format = datetime.strptime(column, '%Y-%m-%d')
    month_format = month_format.strftime("%b-%Y")
    df_icb_processed_pre_referral_sa_pivot = df_icb_processed_pre_referral_sa_pivot.withColumnRenamed(column, month_format)

#2.converting the pivot to pandas dataframe
df_pd_icb_processed_pre_referral_sa_pivot = df_icb_processed_pre_referral_sa_pivot.toPandas()

#3.creating a workbook
ws_total_all = wb['2) Pre referral Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_processed_pre_referral_sa_pivot,
    ws=ws_total_all,
    header=True,
    startrow=64,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_processed_pre_referral_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(64, 108):
        cell_to_copy_from = ws_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_processed_pre_referral_sa.select("EROC_STP_Name").distinct().count()
new_provider = number_of_icb - 42
pre_table_rows = 63 
copy_row = pre_table_rows + 42
end_row = copy_row + new_provider + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D64"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

rules_to_add = []
for rule in ws_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        rules_to_add.append(rule)

for rule in rules_to_add:
    ws_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format

# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_total_all.iter_rows(min_row=64, max_row=106, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
# icb diverted pre referral
from pyspark.sql.functions import col, to_date, lit, sum as F_sum

# Step 0: Ensure EROC_DerMonth is a date
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

# Step 1: Get all ICB
df_all_icb = (
    df_raw_sa  # .filter(col("Acute_Status") == "Acute")
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

# Step 2: Filter for pre-referral SA requests only
df_icb_diverted_pre_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        (col("EROC_DerStatus") != 2) &
        (col("EROC_DerTypeOfSpecialistAdvice") == 1) &
        (col("EROC_DerOutcome").isin([10, 12]))
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

# Step 3: Group and aggregate
df_icb_diverted_pre_referral_sa = (
    df_icb_diverted_pre_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

# Step 4: Cross-join with all orgs and all months to preserve missing combinations

# Get all months in the dataset
all_months_df = (
    df_icb_diverted_pre_referral_sa
    .select("EROC_DerMonth")
    .distinct()
    .filter(col("EROC_DerMonth").isNotNull())
)

# DEBUG: Print all available months
# print(" Available Months (Before Pivot):")
all_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()
# print(all_months)

# Create full grid: all_orgs x all_months
full_grid = (
    df_all_icb.crossJoin(all_months_df)
)

# Left join aggregated data to preserve all org/month combos
df_icb_diverted_pre_referral_sa_full = (
    full_grid.join(
        df_icb_diverted_pre_referral_sa,
        on=["EROC_STP_Code", "EROC_STP_Name", "EROC_DerMonth"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

# Step 5: Pivot table with all valid months
valid_months = all_months  # Already collected above

df_icb_diverted_pre_referral_sa_pivot = (
    df_icb_diverted_pre_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)
#display(df_icb_diverted_pre_referral_sa_pivot)

In [0]:
#1.Outputs of diverted requests for pre referral sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_diverted_pre_referral_sa_pivot.columns[2:]:
    month_format = datetime.strptime(column, '%Y-%m-%d')
    month_format = month_format.strftime("%b-%Y")
    df_icb_diverted_pre_referral_sa_pivot = df_icb_diverted_pre_referral_sa_pivot.withColumnRenamed(column, month_format)

#2.converting the pivot to pandas dataframe
df_pd_icb_diverted_pre_referral_sa_pivot = df_icb_diverted_pre_referral_sa_pivot.toPandas()

#3.creating a workbook
ws_total_all = wb['2) Pre referral Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_diverted_pre_referral_sa_pivot,
    ws=ws_total_all,
    header=True,
    startrow=112,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_diverted_pre_referral_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(112, 156):
        cell_to_copy_from = ws_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_diverted_pre_referral_sa.select("EROC_STP_Name").distinct().count()
new_provider = number_of_icb - 42
pre_table_rows = 111 
copy_row = pre_table_rows + 42
end_row = copy_row + new_provider + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D112"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

rules_to_add = []
for rule in ws_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        rules_to_add.append(rule)

for rule in rules_to_add:
    ws_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format

# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_total_all.iter_rows(min_row=112, max_row=154, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
# Total post referral
from pyspark.sql.functions import col, to_date, lit, sum as F_sum

# Step 0: Ensure EROC_DerMonth is a date
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

# Step 1: Get all Acute providers
df_all_icb = (
    df_raw_sa  # .filter(col("Acute_Status") == "Acute")
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

# Step 2: Filter for post-referral SA requests only
df_icb_total_post_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        # (col("Acute_Status") == "Acute") &
        (col("EROC_DerTypeOfSpecialistAdvice") == 2)# &
        #(col("EROC_DerOutcome") != 40)
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

# Step 3: Group and aggregate
df_icb_total_post_referral_sa = (
    df_icb_total_post_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

# Step 4: Cross-join with all orgs and all months to preserve missing combinations

# Get all months in the dataset
all_months_df = (
    df_icb_total_post_referral_sa
    .select("EROC_DerMonth")
    .distinct()
    .filter(col("EROC_DerMonth").isNotNull())
)

# DEBUG: Print all available months
# print(" Available Months (Before Pivot):")
all_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()
# print(all_months)

# Create full grid: all_orgs x all_months
full_grid = (
    df_all_icb.crossJoin(all_months_df)
)

# Left join aggregated data to preserve all org/month combos
df_icb_total_post_referral_sa_full = (
    full_grid.join(
        df_icb_total_post_referral_sa,
        on=["EROC_STP_Code", "EROC_STP_Name", "EROC_DerMonth"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

# Step 5: Pivot table with all valid months
valid_months = all_months  # Already collected above

df_icb_total_post_referral_sa_pivot = (
    df_icb_total_post_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)
#display(df_icb_total_post_referral_sa_pivot)

In [0]:
#1.Outputs of Total requests for all post referral sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_total_post_referral_sa_pivot.columns[2:]:
    try:
        month_format = datetime.strptime(column, '%Y-%m-%d')
        month_format = month_format.strftime("%b-%Y")
        df_icb_total_post_referral_sa_pivot = df_icb_total_post_referral_sa_pivot.withColumnRenamed(column, month_format)
    except ValueError:
        continue

#2.converting the pivot to pandas dataframe
df_pd_icb_total_post_referral_sa_pivot = df_icb_total_post_referral_sa_pivot.toPandas()

#3.creating a workbook
ws_icb_total_all = wb['3) Post referral Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_total_post_referral_sa_pivot,
    ws=ws_icb_total_all,
    header=True,
    startrow=15,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_total_post_referral_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(15, 59):
        cell_to_copy_from = ws_icb_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_total_post_referral_sa.select("EROC_STP_Name").distinct().count()
new_icb = number_of_icb - 42
pre_table_rows = 14 
copy_row = pre_table_rows + 42
end_row = copy_row + new_icb + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_icb_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D15"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

for rule in ws_icb_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        ws_icb_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_icb_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_icb_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format
        
# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_icb_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_icb_total_all.iter_rows(min_row=15, max_row=57, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
# icb processed post referral
from pyspark.sql.functions import col, to_date, lit, sum as F_sum

# Step 0: Ensure EROC_DerMonth is a date
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

# Step 1: Get all Acute providers
df_all_icb = (
    df_raw_sa  # .filter(col("Acute_Status") == "Acute")
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

# Step 2: Filter for pre-referral SA requests only
df_icb_processed_post_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        (col("EROC_DerStatus") != 2) &
        (col("EROC_DerTypeOfSpecialistAdvice") == 2) &
        (col("EROC_DerOutcome") != 40)
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

# Step 3: Group and aggregate
df_icb_processed_post_referral_sa = (
    df_icb_processed_post_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

# Step 4: Cross-join with all orgs and all months to preserve missing combinations

# Get all months in the dataset
all_months_df = (
    df_icb_processed_post_referral_sa
    .select("EROC_DerMonth")
    .distinct()
    .filter(col("EROC_DerMonth").isNotNull())
)

# DEBUG: Print all available months
# print(" Available Months (Before Pivot):")
all_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()
# print(all_months)

# Create full grid: all_orgs x all_months
full_grid = (
    df_all_icb.crossJoin(all_months_df)
)

# Left join aggregated data to preserve all org/month combos
df_icb_processed_post_referral_sa_full = (
    full_grid.join(
        df_icb_processed_post_referral_sa,
        on=["EROC_STP_Code", "EROC_STP_Name", "EROC_DerMonth"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

# Step 5: Pivot table with all valid months
valid_months = all_months  # Already collected above

df_icb_processed_post_referral_sa_pivot = (
    df_icb_processed_post_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)
#display(df_icb_processed_post_referral_sa_pivot)

In [0]:
#1.Outputs of processed requests for post referral sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_processed_post_referral_sa_pivot.columns[2:]:
    month_format = datetime.strptime(column, '%Y-%m-%d')
    month_format = month_format.strftime("%b-%Y")
    df_icb_processed_post_referral_sa_pivot = df_icb_processed_post_referral_sa_pivot.withColumnRenamed(column, month_format)

#2.converting the pivot to pandas dataframe
df_pd_icb_processed_post_referral_sa_pivot = df_icb_processed_post_referral_sa_pivot.toPandas()

#3.creating a workbook
ws_total_all = wb['3) Post referral Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_processed_post_referral_sa_pivot,
    ws=ws_total_all,
    header=True,
    startrow=64,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_processed_post_referral_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(64, 108):
        cell_to_copy_from = ws_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_processed_post_referral_sa.select("EROC_STP_Name").distinct().count()
new_provider = number_of_icb - 42
pre_table_rows = 63 
copy_row = pre_table_rows + 42
end_row = copy_row + new_provider + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D64"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

rules_to_add = []
for rule in ws_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        rules_to_add.append(rule)

for rule in rules_to_add:
    ws_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format

# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_total_all.iter_rows(min_row=64, max_row=106, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
# icb diverted post referral
from pyspark.sql.functions import col, to_date, lit, sum as F_sum

# Step 0: Ensure EROC_DerMonth is a date
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

# Step 1: Get all ICB
df_all_icb = (
    df_raw_sa  # .filter(col("Acute_Status") == "Acute")
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

# Step 2: Filter for pre-referral SA requests only
df_icb_diverted_post_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        (col("EROC_DerStatus") != 2) &
        (col("EROC_DerTypeOfSpecialistAdvice") == 2) &
        (col("EROC_DerOutcome").isin([10, 12]))
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

# Step 3: Group and aggregate
df_icb_diverted_post_referral_sa = (
    df_icb_diverted_post_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

# Step 4: Cross-join with all orgs and all months to preserve missing combinations

# Get all months in the dataset
all_months_df = (
    df_icb_diverted_post_referral_sa
    .select("EROC_DerMonth")
    .distinct()
    .filter(col("EROC_DerMonth").isNotNull())
)

# DEBUG: Print all available months
# print(" Available Months (Before Pivot):")
all_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()
# print(all_months)

# Create full grid: all_orgs x all_months
full_grid = (
    df_all_icb.crossJoin(all_months_df)
)

# Left join aggregated data to preserve all org/month combos
df_icb_diverted_post_referral_sa_full = (
    full_grid.join(
        df_icb_diverted_post_referral_sa,
        on=["EROC_STP_Code", "EROC_STP_Name", "EROC_DerMonth"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

# Step 5: Pivot table with all valid months
valid_months = all_months  # Already collected above

df_icb_diverted_post_referral_sa_pivot = (
    df_icb_diverted_post_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)
#display(df_icb_diverted_post_referral_sa_pivot)

In [0]:
#1.Outputs of diverted requests for post referral sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_diverted_post_referral_sa_pivot.columns[2:]:
    month_format = datetime.strptime(column, '%Y-%m-%d')
    month_format = month_format.strftime("%b-%Y")
    df_icb_diverted_post_referral_sa_pivot = df_icb_diverted_post_referral_sa_pivot.withColumnRenamed(column, month_format)

#2.converting the pivot to pandas dataframe
df_pd_icb_diverted_post_referral_sa_pivot = df_icb_diverted_post_referral_sa_pivot.toPandas()

#3.creating a workbook
ws_total_all = wb['3) Post referral Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_diverted_post_referral_sa_pivot,
    ws=ws_total_all,
    header=True,
    startrow=112,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_diverted_post_referral_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(112, 156):
        cell_to_copy_from = ws_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_diverted_post_referral_sa.select("EROC_STP_Name").distinct().count()
new_provider = number_of_icb - 42
pre_table_rows = 111 
copy_row = pre_table_rows + 42
end_row = copy_row + new_provider + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D112"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

rules_to_add = []
for rule in ws_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        rules_to_add.append(rule)

for rule in rules_to_add:
    ws_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format

# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_total_all.iter_rows(min_row=112, max_row=154, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
#Total other referral SA

from pyspark.sql.functions import col, to_date, lit, sum as F_sum

#  Step 0: Ensure date column is properly typed
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

#  Step 1: All Acute orgs
df_all_icb = (
    df_raw_sa#.filter(col("Acute_Status") == "Acute")
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

#  Step 2: Filter for type 99
df_icb_total_other_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        #(col("Acute_Status") == "Acute") &
        (col("EROC_DerTypeOfSpecialistAdvice") == 99)
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

#  Step 3: Aggregate
df_icb_total_other_referral_sa = (
    df_icb_total_other_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

#  Step 4: Get all valid months in data
all_months_df = (
    df_icb_total_other_referral_sa
    .select("EROC_DerMonth")
    .distinct()
    .filter(col("EROC_DerMonth").isNotNull())
)

#  DEBUG: Show all months
#print(" Available Months for Type 99:")
valid_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()
#print(valid_months)

#  Step 5: Build org × month grid
full_grid = df_all_icb.crossJoin(all_months_df)

#  Step 6: Join actual data to full grid
df_icb_total_other_referral_sa_full = (
    full_grid.join(
        df_icb_total_other_referral_sa,
        on=["EROC_STP_Code", "EROC_STP_Name", "EROC_DerMonth"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

#  Step 7: Pivot table with all valid months
df_icb_total_other_referral_sa_pivot = (
    df_icb_total_other_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)
#display(df_icb_total_other_referral_sa_pivot)

In [0]:
#1.Outputs of Total requests for all other referral sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_total_other_referral_sa_pivot.columns[2:]:
    try:
        month_format = datetime.strptime(column, '%Y-%m-%d')
        month_format = month_format.strftime("%b-%Y")
        df_icb_total_other_referral_sa_pivot = df_icb_total_other_referral_sa_pivot.withColumnRenamed(column, month_format)
    except ValueError:
        continue

#2.converting the pivot to pandas dataframe
df_pd_icb_total_other_referral_sa_pivot = df_icb_total_other_referral_sa_pivot.toPandas()

#3.creating a workbook
ws_icb_total_all = wb['4) Other Types of Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_total_other_referral_sa_pivot,
    ws=ws_icb_total_all,
    header=True,
    startrow=15,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_total_other_referral_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(15, 59):
        cell_to_copy_from = ws_icb_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_total_other_referral_sa.select("EROC_STP_Name").distinct().count()
new_icb = number_of_icb - 42
pre_table_rows = 14 
copy_row = pre_table_rows + 42
end_row = copy_row + new_icb + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_icb_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D15"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

for rule in ws_icb_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        ws_icb_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_icb_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_icb_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format
        
# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_icb_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_icb_total_all.iter_rows(min_row=15, max_row=57, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
# icb processed other referral
from pyspark.sql.functions import col, to_date, lit, sum as F_sum

# Step 0: Ensure EROC_DerMonth is a date
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

# Step 1: Get all ICB
df_all_icb = (
    df_raw_sa  # .filter(col("Acute_Status") == "Acute")
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

# Step 2: Filter for pre-referral SA requests only
df_icb_processed_other_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        (col("EROC_DerStatus") != 2) &
        (col("EROC_DerTypeOfSpecialistAdvice") == 99) &
        (col("EROC_DerOutcome") != 40)
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

# Step 3: Group and aggregate
df_icb_processed_other_referral_sa = (
    df_icb_processed_other_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

# Step 4: Cross-join with all orgs and all months to preserve missing combinations

# Get all months in the dataset
all_months_df = (
    df_icb_processed_other_referral_sa
    .select("EROC_DerMonth")
    .distinct()
    .filter(col("EROC_DerMonth").isNotNull())
)

# DEBUG: Print all available months
# print(" Available Months (Before Pivot):")
all_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()
# print(all_months)

# Create full grid: all_orgs x all_months
full_grid = (
    df_all_icb.crossJoin(all_months_df)
)

# Left join aggregated data to preserve all org/month combos
df_icb_processed_other_referral_sa_full = (
    full_grid.join(
        df_icb_processed_other_referral_sa,
        on=["EROC_STP_Code", "EROC_STP_Name", "EROC_DerMonth"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

# Step 5: Pivot table with all valid months
valid_months = all_months  # Already collected above

df_icb_processed_other_referral_sa_pivot = (
    df_icb_processed_other_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)
#display(df_icb_processed_other_referral_sa_pivot)

In [0]:
#1.Outputs of processed requests for other referral sa formatting date headers along the pivot
from datetime import datetime

for column in df_icb_processed_other_referral_sa_pivot.columns[2:]:
    month_format = datetime.strptime(column, '%Y-%m-%d')
    month_format = month_format.strftime("%b-%Y")
    df_icb_processed_other_referral_sa_pivot = df_icb_processed_other_referral_sa_pivot.withColumnRenamed(column, month_format)

#2.converting the pivot to pandas dataframe
df_pd_icb_processed_other_referral_sa_pivot = df_icb_processed_other_referral_sa_pivot.toPandas()

#3.creating a workbook
ws_total_all = wb['4) Other Types of Spec Advice']

excel.insert_pandas_df_into_excel(
    df=df_pd_icb_processed_other_referral_sa_pivot,
    ws=ws_total_all,
    header=True,
    startrow=64,
    startcol=2,
    index=False,
)

#4.Check width of table and get column numbers of unformatted columns 
number_of_months = df_icb_processed_other_referral_sa.select("EROC_DerMonth").distinct().count()
new_months = number_of_months - 37
pre_date_columns = 3
copy_column = 37 + pre_date_columns
end_column = number_of_months + pre_date_columns + 1

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(64, 108):
        cell_to_copy_from = ws_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_processed_other_referral_sa.select("EROC_STP_Name").distinct().count()
new_provider = number_of_icb - 42
pre_table_rows = 63 
copy_row = pre_table_rows + 42
end_row = copy_row + new_provider + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D64"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

rules_to_add = []
for rule in ws_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        rules_to_add.append(rule)

for rule in rules_to_add:
    ws_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format

# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_total_all.iter_rows(min_row=64, max_row=106, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"



In [0]:
#Diverted other referral SA

from pyspark.sql.functions import col, to_date, lit, sum as F_sum
from datetime import datetime
from pyspark.sql import Row

# Step 0: Ensure EROC_DerMonth is a date
df_raw_sa = df_raw_sa.withColumn("EROC_DerMonth", to_date("EROC_DerMonth"))

# Step 1: Get all ICBs
df_all_icb = (
    df_raw_sa
    .select("EROC_STP_Code", "EROC_STP_Name")
    .distinct()
)

# Step 2: Filter for pre-referral SA requests only
df_icb_diverted_other_referral_sa = (
    df_raw_sa.filter(
        (col("EROC_DerMonth") > lit("2022-03-01")) &
        (col("EROC_DerStatus") != 2) &
        (col("EROC_DerTypeOfSpecialistAdvice") == 99) &
        (col("EROC_DerOutcome").isin([10, 12]))
    )
    .select("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name", "EROC_Requests")
)

# Step 3: Group and aggregate
df_icb_agg = (
    df_icb_diverted_other_referral_sa
    .groupby("EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name")
    .agg(F_sum("EROC_Requests").alias("Requests"))
)

#Step 4: Ensure full month range is included ===

# 4.1 Define start and end dates
start_date = datetime(2022, 4, 1)
end_date = datetime.combine(df_raw_sa.agg({"EROC_DerMonth": "max"}).collect()[0][0], datetime.min.time())

# 4.2 Generate full list of first-of-months from start to end
def generate_months(start, end):
    months = []
    current = start.replace(day=1)
    while current <= end:
        months.append(Row(EROC_DerMonth=current))
        if current.month == 12:
            current = current.replace(year=current.year + 1, month=1)
        else:
            current = current.replace(month=current.month + 1)
    return months

month_rows = generate_months(start_date, end_date)
all_months_df = spark.createDataFrame(month_rows)

# 4.3 Create full grid: all_orgs x all_months
full_grid = df_all_icb.crossJoin(all_months_df)

# 4.4 Left join actual data to preserve all org/month combinations
df_icb_diverted_other_referral_sa_full = (
    full_grid.join(
        df_icb_agg,
        on=["EROC_DerMonth", "EROC_STP_Code", "EROC_STP_Name"],
        how="left"
    )
    .fillna(0, subset=["Requests"])
)

# 4.5 Collect valid months for pivoting
valid_months = all_months_df.orderBy("EROC_DerMonth").rdd.flatMap(lambda x: x).collect()

# Step 5: Pivot table with all valid months
df_icb_diverted_other_referral_sa_pivot = (
    df_icb_diverted_other_referral_sa_full
    .groupby("EROC_STP_Code", "EROC_STP_Name")
    .pivot("EROC_DerMonth", values=valid_months)
    .agg(F_sum("Requests"))
    .orderBy("EROC_STP_Name", "EROC_STP_Code")
    .withColumnRenamed("EROC_STP_Code", "ICB Code")
    .withColumnRenamed("EROC_STP_Name", "ICB Name")
)

#display(df_icb_diverted_other_referral_sa_pivot)

In [0]:
# 1. Outputs of diverted requests for all other referral sa – formatting date headers in the pivot
from datetime import datetime
import pandas as pd  # Needed for (to_datetime)

# Rename date columns robustly
for column in df_icb_diverted_other_referral_sa_pivot.columns[2:]:
    try:
        # Handle both string and datetime column headers
        if isinstance(column, datetime):
            month_format = column.strftime("%b-%Y")
        else:
            month_format = pd.to_datetime(column).strftime("%b-%Y")
        df_icb_diverted_other_referral_sa_pivot = df_icb_diverted_other_referral_sa_pivot.withColumnRenamed(column, month_format)
    except Exception as e:
        print(f"Could not rename column {column}: {e}")
        continue

# Optional: check final column headers, used to help debug
#print("Final column headers:", df_icb_diverted_other_referral_sa_pivot.columns)

# 2. Converting the pivot to pandas dataframe
df_pd_icb_diverted_other_referral_sa_pivot = df_icb_diverted_other_referral_sa_pivot.toPandas()

# 2a Ensure the first month column is numeric (so SUM works), 
#     without converting counts to strings
import pandas as pd  # Needed for pd.notnull

# Adjust this index if needed
date_column_name = df_pd_icb_diverted_other_referral_sa_pivot.columns[2]

def clean_first_month_value(x):
    # If it's a datetime, just return the original datetime
    if pd.notnull(x) and hasattr(x, 'strftime'):
        return x
    # If it's already a number, keep it
    if isinstance(x, (int, float)):
        return x
    # If it's blank/NaN, treat as zero
    if pd.isnull(x) or str(x).strip() in ("", "-", "—"):
        return 0
    # Try to coerce strings like "123" into numbers
    try:
        return float(str(x).replace(",", ""))
    except ValueError:
        return 0

# Apply cleaning
df_pd_icb_diverted_other_referral_sa_pivot[date_column_name] = \
    df_pd_icb_diverted_other_referral_sa_pivot[date_column_name].apply(clean_first_month_value)


# 3. Creating a workbook
ws_icb_total_all = wb['4) Other Types of Spec Advice']

# 4. Insert the formatted DataFrame into the Excel worksheet
excel.insert_pandas_df_into_excel(
    df=df_pd_icb_diverted_other_referral_sa_pivot,
    ws=ws_icb_total_all,
    header=True,
    startrow=112,
    startcol=2,
    index=False,
)

#5.copy and paste formatting onto unformatted columns
for column_number in range(copy_column, end_column):
    for row_number in range(112, 157):
        cell_to_copy_from = ws_icb_total_all.cell(row=row_number, column=copy_column)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#6.get height of table and get row numbers of unformatted rows
number_of_icb = df_icb_diverted_other_referral_sa.select("EROC_STP_Name").distinct().count()
new_icb = number_of_icb - 42
pre_table_rows = 111 
copy_row = pre_table_rows + 42
end_row = copy_row + new_icb + 1

#7.copy and paste formatting onto unformatted rows
for row_number in range(copy_row, end_row):
    for column_number in range(2, end_column):
        cell_to_copy_from = ws_icb_total_all.cell(row=copy_row, column=column_number)
        cell_to_paste_to = ws_icb_total_all.cell(row=row_number, column=column_number)
        excel.copy_all_cell_styles(cell_to_copy_from, cell_to_paste_to)
        
#8.Use the values calculated above to get the cell range of the whole table
conditional_formatting_start_cell = "D112"
conditional_formatting_end_col = openpyxl.utils.cell.get_column_letter(end_column - 1)
conditional_formatting_end_row = end_row - 1
conditional_formatting_end_cell = conditional_formatting_end_col + str(conditional_formatting_end_row)
conditional_formatting_range = f"{conditional_formatting_start_cell}:{conditional_formatting_end_cell}"

#9.Copy the existing conditional formatting rule, but make it cover the whole table using the range created above.
from openpyxl.formatting.rule import Rule

for rule in ws_icb_total_all.conditional_formatting:
    if isinstance(rule, Rule):
        ws_icb_total_all.conditional_formatting.add(conditional_formatting_range, rule)

#10.updating publishing date header
ws_icb_total_all.cell(row=3, column=3).value = date_header

from openpyxl.styles import NamedStyle

#11.Define the number format style
number_style = NamedStyle(name="number", number_format="#,##0")

#12.Apply the number format to the specified range
for row in ws_icb_total_all.iter_rows(min_row=16, max_row=end_row, min_col=pre_date_columns + 1, max_col=end_column):
    for cell in row:
        cell.number_format = number_style.number_format
        
# Apply the number format to specific cells
for row in [58, 107, 155]:
    ws_icb_total_all.cell(row=row, column=4).number_format = number_style.number_format

# Prevent overwriting the date format in the specified range
for row in ws_icb_total_all.iter_rows(min_row=112, max_row=157, min_col=4, max_col=end_column):
    for cell in row:
        if isinstance(cell.value, datetime):
            cell.number_format = "mmm-yy"

In [0]:
#number formatting

from openpyxl.styles import NamedStyle

# Create custom style that shows dash for 0
dash_number_format = NamedStyle(name="dash_format", number_format='#,##0;-#,##0;"-"')

# Avoid error if style already exists
if "dash_format" not in wb.named_styles:
    wb.add_named_style(dash_number_format)

# Apply this formatting across all relevant sheets and ranges
target_sheet = wb["4) Other Types of Spec Advice"]

for row in target_sheet.iter_rows(min_row=16, max_row=target_sheet.max_row, min_col=3, max_col=target_sheet.max_column):
    for cell in row:
        if cell.value is None or cell.value == 0:
            cell.number_format = dash_number_format.number_format

# Ensure column 4, rows 113-154 use the dash format for 0s (and numbers)
for row in target_sheet.iter_rows(min_row=113, max_row=154, min_col=4, max_col=4):
    for cell in row:
        cell.number_format = dash_number_format.number_format

In [0]:
wb.save('Outputs/Specialist advice activity by ICB from April 2022.xlsx')